## Calculating Audio Durations and Updating CSV

This Jupyter Notebook calculates the durations of audio files and updates a CSV file. The CSV file, named `audio_annotations.csv`, is expected to have information about audio files, including paths.

1. **Read the CSV File**: The code reads the input CSV file, which is assumed to be structured with columns for paths and other attributes.

2. **Sort by Path**: The audio files are sorted by their paths to group identical audio files together for efficient duration calculations.

3. **Calculate Durations**: It iterates through the sorted DataFrame and calculates the duration for each audio file using the `pydub` library.

4. **Update the DataFrame**: The code updates the DataFrame with the calculated audio durations. If an audio file shares a path with a previous one, it reuses the previously calculated duration to save computation time.

5. **Rearrange Columns**: The columns are rearranged to place the new 'audio_duration' column next to the 'time' column.

6. **Save to New CSV**: The updated DataFrame is saved as a new CSV file.


In [1]:
import pandas as pd
from pydub import AudioSegment

In [2]:
ROOT_PATH = "../../../desarrollo/"

DATASET_FOLDER = ROOT_PATH + "Data/Dataset/Audios/"

# Load the CSV file
input_file = ROOT_PATH + "Data/Annotations/" + "d00_audio_annotations.csv"
df = pd.read_csv(input_file)

# Path to the folder where you want to save the CSV files
output_file = ROOT_PATH + "Data/Annotations/" + "d01_audio_annotations.csv"

In [3]:
# Leer el archivo CSV
df = pd.read_csv(input_file)

In [6]:
# Sort the DataFrame by 'path'
df = df.sort_values(by='path')

# Look for files that are in the csv but not in the folder and print path
for index, row in df.iterrows():
    file_path = row["path"]
    file_path = DATASET_FOLDER + file_path
    try:
        audio = AudioSegment.from_file(file_path)
    except FileNotFoundError:
        print(file_path)

In [5]:
# Delete rows with files that are not in the folder
df = df.drop(df[df['path'] == 'AM10/2023_05_09/AM10_20230509_093000.WAV'].index)

In [7]:
# Sort the DataFrame by 'path'
df = df.sort_values(by='path')

# Create a new column to store the audio duration in HH:MM:SS format
df['audio_duration'] = ""

# Initialize variables to track the previous path and its audio duration
prev_path = ""
prev_duration = None

# Iterate through the sorted DataFrame and calculate the duration of each audio
for index, row in df.iterrows():
    path = row['path']
    if path != prev_path:
        try:
            # This is a new audio file, calculate its duration
            audio = AudioSegment.from_file(DATASET_FOLDER + path, format="wav")
            duration_seconds = len(audio) / 1000  # Convert to seconds
            duration_time = pd.to_datetime(duration_seconds, unit='s').strftime('%H:%M:%S')
            df.at[index, 'audio_duration'] = duration_time
            prev_duration = duration_time
        except FileNotFoundError:
            print(file_path)
    else:
        # This audio file has the same path as the previous one, use the previous duration
        df.at[index, 'audio_duration'] = prev_duration

    prev_path = path

# Rearrange the columns to place 'audio_duration' after 'time'
columns = ['path', 'annotator', 'recorder', 'date', 'time', 'audio_duration', 'start_time', 'end_time', 'low_frequency', 'high_frequency', 'specie']
#columns = ['path', 'recorder', 'date', 'time', 'audio_duration', 'start_time', 'end_time', 'specie']
df = df[columns]

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

In [8]:
df

,path,annotator,recorder,date,time,audio_duration,start_time,end_time,low_frequency,high_frequency,specie
2706,AM1/2023_05_10/AM1_20230510_060000.WAV,Edu,AM1,2023/05/10,06:00:00,00:01:00,12.488571,13.105714,3003.168457,5366.754395,galerida theklae
2703,AM1/2023_05_10/AM1_20230510_060000.WAV,Edu,AM1,2023/05/10,06:00:00,00:01:00,11.331429,12.488571,3583.403809,6542.934570,abiotic noise
2702,AM1/2023_05_10/AM1_20230510_060000.WAV,Edu,AM1,2023/05/10,06:00:00,00:01:00,11.297143,13.474286,-1.000000,1754.262451,abiotic noise
2701,AM1/2023_05_10/AM1_20230510_060000.WAV,Edu,AM1,2023/05/10,06:00:00,00:01:00,6.960000,11.297143,18.972063,5970.894531,abiotic noise
2700,AM1/2023_05_10/AM1_20230510_060000.WAV,Edu,AM1,2023/05/10,06:00:00,00:01:00,5.914286,6.925714,1881.718994,3449.984619,cyanopica cooki
...,...,...,...,...,...,...,...,...,...,...,...
3018,AM8/2023_05_30/AM8_20230530_110000.WAV,Giulia,AM8,2023/05/30,11:00:00,00:01:00,15.326958,19.618950,7534.428564,16000.000000,carduelis carduelis
3016,AM8/2023_05_30/AM8_20230530_110000.WAV,Giulia,AM8,2023/05/30,11:00:00,00:01:00,10.070100,11.012785,10045.170898,15652.996094,carduelis carduelis
3017,AM8/2023_05_30/AM8_20230530_110000.WAV,Giulia,AM8,2023/05/30,11:00:00,00:01:00,14.339911,16.039736,1607.101094,3234.563699,luscinia megarrhyncos
3022,AM8/2023_05_30/AM8_20230530_110000.WAV,Giulia,AM8,2023/05/30,11:00:00,00:01:00,48.498399,51.803344,11486.522738,14348.650436,carduelis carduelis
